# 🔗 Graph Neural Networks: GCN, GAT & Applications

Deep learning on graph-structured data.

## Learning Outcomes
- Graph representation basics
- Node classification with GCN/GAT
- Graph classification for molecules
- Recommendation with graph embeddings

**Level**: Advanced | **Time**: 75 min | **GPU**: Recommended

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. Graph Basics

In [ ]:
# Create a simple graph
# Nodes: 0, 1, 2, 3, 4
# Edges: 0-1, 0-2, 1-2, 1-3, 2-4, 3-4

edge_index = torch.tensor([
    [0, 0, 1, 1, 2, 3],  # Source nodes
    [1, 2, 2, 3, 4, 4]   # Target nodes
], dtype=torch.long)

# Make undirected
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)

# Node features (5 nodes, 3 features each)
x = torch.randn(5, 3)

# Node labels (for classification)
y = torch.tensor([0, 0, 1, 1, 1])

print(f"Nodes: 5, Edges: {edge_index.shape[1]}")
print(f"Node features shape: {x.shape}")

## 2. Graph Convolutional Network (GCN)

In [ ]:
class GCNLayer(nn.Module):
    """Simple GCN layer."""
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, x, edge_index):
        # Add self-loops
        num_nodes = x.size(0)
        self_loops = torch.stack([torch.arange(num_nodes), torch.arange(num_nodes)])
        edge_index = torch.cat([edge_index, self_loops.to(edge_index.device)], dim=1)
        
        # Compute degree
        row, col = edge_index
        deg = torch.zeros(num_nodes).scatter_add_(0, col, torch.ones(edge_index.size(1)))
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        
        # Normalize
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        # Aggregate
        out = torch.zeros_like(x)
        for i in range(edge_index.size(1)):
            out[col[i]] += norm[i] * x[row[i]]
        
        return self.linear(out)

class GCN(nn.Module):
    def __init__(self, in_features, hidden, num_classes):
        super().__init__()
        self.conv1 = GCNLayer(in_features, hidden)
        self.conv2 = GCNLayer(hidden, num_classes)
    
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

gcn = GCN(3, 16, 2)
print(f"GCN Parameters: {sum(p.numel() for p in gcn.parameters()):,}")

## 3. Training GCN

In [ ]:
# Training
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01, weight_decay=5e-4)
train_mask = torch.tensor([True, True, True, False, False])  # Train on first 3 nodes

print("Training GCN...")
for epoch in range(50):
    gcn.train()
    optimizer.zero_grad()
    out = gcn(x, edge_index)
    loss = F.nll_loss(out[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        gcn.eval()
        pred = out.argmax(dim=1)
        acc = (pred == y).float().mean()
        print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc.item():.4f}")

## 4. PyTorch Geometric (PyG)

In [ ]:
try:
    from torch_geometric.nn import GCNConv, GATConv, SAGEConv
    from torch_geometric.data import Data
    from torch_geometric.datasets import Planetoid
    
    # Load Cora dataset
    dataset = Planetoid(root='./data', name='Cora')
    data = dataset[0]
    
    print(f"\n📊 Cora Dataset:")
    print(f"  Nodes: {data.num_nodes}")
    print(f"  Edges: {data.num_edges}")
    print(f"  Features: {data.num_features}")
    print(f"  Classes: {dataset.num_classes}")
    
    # PyG GCN
    class PyGGCN(nn.Module):
        def __init__(self, num_features, hidden, num_classes):
            super().__init__()
            self.conv1 = GCNConv(num_features, hidden)
            self.conv2 = GCNConv(hidden, num_classes)
        
        def forward(self, data):
            x, edge_index = data.x, data.edge_index
            x = F.relu(self.conv1(x, edge_index))
            x = F.dropout(x, p=0.5, training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
    
    model = PyGGCN(dataset.num_features, 16, dataset.num_classes)
    print(f"\n  PyG GCN ready!")
except ImportError:
    print("Install PyG: pip install torch-geometric")

## 5. Graph Attention Network (GAT)

In [ ]:
class SimpleGAT(nn.Module):
    """Simplified Graph Attention Layer."""
    def __init__(self, in_features, out_features, heads=4):
        super().__init__()
        self.heads = heads
        self.linear = nn.Linear(in_features, out_features * heads)
        self.attention = nn.Linear(2 * out_features, 1)
        self.out_features = out_features
    
    def forward(self, x, edge_index):
        h = self.linear(x).view(-1, self.heads, self.out_features)
        # Simplified: just average heads
        return h.mean(dim=1)

gat = SimpleGAT(3, 16, heads=4)
out = gat(x, edge_index)
print(f"GAT output shape: {out.shape}")

## 6. Applications

In [ ]:
import pandas as pd

applications = pd.DataFrame({
    'Application': ['Social Networks', 'Molecules', 'Knowledge Graphs', 'Recommendations', 'Traffic'],
    'Task': ['Node Classification', 'Graph Classification', 'Link Prediction', 'Link Prediction', 'Forecasting'],
    'Model': ['GCN/GAT', 'GIN/MPNN', 'TransE/RotatE', 'LightGCN', 'STGCN'],
    'Example': ['User interests', 'Drug discovery', 'Entity relations', 'User-item', 'Speed prediction']
})

print("📊 GNN Applications:")
display(applications)

## 7. Model Comparison

In [ ]:
comparison = pd.DataFrame({
    'Model': ['GCN', 'GAT', 'GraphSAGE', 'GIN', 'TransformerConv'],
    'Cora Acc': ['81.5%', '83.0%', '79.0%', '77.6%', '84.2%'],
    'Aggregation': ['Sum', 'Attention', 'Sample', 'Sum + MLP', 'Attention'],
    'Inductive': ['No', 'No', 'Yes', 'Yes', 'No'],
    'Best For': ['General', 'Heterophily', 'Large graphs', 'Graph class', 'Complex']
})

print("📊 GNN Model Comparison:")
display(comparison)

## 🎯 Key Takeaways
1. GNN = message passing between nodes
2. GAT uses attention for edge weights
3. GraphSAGE enables inductive learning
4. Over-smoothing limits depth

## 📚 Further Reading
- Kipf & Welling, "Semi-Supervised Classification with GCNs" (2017)
- Veličković et al., "Graph Attention Networks" (2018)
- Hamilton et al., "Inductive Representation Learning" (GraphSAGE, 2017)